In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llmgamma/gamma_dataset.json


In [2]:
import json, pprint

with open("/kaggle/input/llmgamma/gamma_dataset.json", "r") as f:
    data = json.load(f)

pprint.pprint(data[:3])


[{'input': 'Hey Radhika! Kaisi ho?',
  'output': 'Hey Rohan, main bilkul thik hun. Tu bata, kaisa hai?'},
 {'input': "I'm good too, yaar. Tumne suna ki next week school wali trip hai?",
  'output': 'Haan, mujhe toh pata chal gaya hai. Itna excited hun mai! Tu bhi '
            'aa raha hai na?'},
 {'input': 'Bilkul, Radhika. Ek trip miss karunga toh behad boring ho jayega. '
           'Hum kaha ja rahe hai?',
  'output': 'Humare college ne Goa ka plan banaya hai. Wahin par enjoy karne '
            'wale hai, khub saare beaches aur parties.'}]


In [2]:
!pip install transformers peft datasets accelerate sentencepiece --quiet


In [3]:
import json

path = "/kaggle/input/llmgamma/gamma_dataset.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

print("Total training samples:", len(data))
data[:3]   # preview


Total training samples: 4458


[{'input': 'Hey Radhika! Kaisi ho?',
  'output': 'Hey Rohan, main bilkul thik hun. Tu bata, kaisa hai?'},
 {'input': "I'm good too, yaar. Tumne suna ki next week school wali trip hai?",
  'output': 'Haan, mujhe toh pata chal gaya hai. Itna excited hun mai! Tu bhi aa raha hai na?'},
 {'input': 'Bilkul, Radhika. Ek trip miss karunga toh behad boring ho jayega. Hum kaha ja rahe hai?',
  'output': 'Humare college ne Goa ka plan banaya hai. Wahin par enjoy karne wale hai, khub saare beaches aur parties.'}]

In [4]:
from datasets import Dataset

dataset = Dataset.from_list(data)
dataset


Dataset({
    features: ['input', 'output'],
    num_rows: 4458
})

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/phi-2"   # Kaggle me Phi-3.5 available nahi, Phi-2 finetune karo (works perfectly for flirty GF)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def format_example(example):
    user = example["input"]
    answer = example["output"]
    text = f"<user>: {user}\n<assistant>: {answer}"
    return {"text": text}

dataset = dataset.map(format_example)
dataset


Map:   0%|          | 0/4458 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 4458
})

In [8]:
# Fix padding issue
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


In [9]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_ds = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/4458 [00:00<?, ? examples/s]

In [10]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 7,864,320 || all params: 2,787,548,160 || trainable%: 0.2821


In [23]:
import torch
model = model.to("cuda:0")  # FORCE SINGLE GPU


In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./phi2-gf-chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    save_steps=200,
    num_train_epochs=1,
    learning_rate=2e-4,
    
    # IMPORTANT FIX (bnb hata diya)
    optim="adamw_torch",

    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)


In [21]:
import torch
model = model.to("cuda:0")  # FORCE SINGLE GPU


In [25]:
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 102.00 MiB. GPU 1 has a total capacity of 14.56 GiB of which 51.81 MiB is free. Including non-PyTorch memory, this process has 14.51 GiB memory in use. Of the allocated memory 14.14 GiB is allocated by PyTorch, and 183.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)